In [ ]:
#|default_exp downloader

# It all starts with FAIR data 

> Towards cloud optimized analysis-ready data 

In the world of open heritage science, there is much ambition to share research data. In other words to make data FAIR: Findable, Accessible, Interoperable and Reusable. In practice however, it turns out to be extremely difficult to make this happen for a data intensive project like the Falnama project. Without analyzing this problem in detail we find that two major obstacles are the sheer size of all our data and the impossibility for a divers team of researchers to draw conclusions from the raw data directly. 

At this point in time, we can not solve this for the dozen different types of data that we collected in the Falnama project. As a first step we will focus here on a the analysis of a single type of data that we have looked at so far: our Macro XRF scanning data that was obtained thanks to the courtesy of Klassik Stiftung Weimar.  

## Downloading the Falnama data 

:::(.callout-note} For now we have implemented access to our MA-XRF data only. Our intention is to expand functionality to other datasets in the near future. :::

To start working with the data import `Falnama_maxrf_data`. 

In [ ]:
from falnama import Falnama_maxrf_data 

To list which data filenames are included in this dataset, initialize and use the `.show_registry_files()`. 

In [ ]:
maxrf_data = Falnama_maxrf_data()
maxrf_data.show_registry_files()

['WM-71803-01_400_600_50.datastack',
 'WM-71803-03_250_300_50.datastack',
 'WM-71803-08_250_300_50.datastack',
 'WM-71803-10_250_300_50.datastack',
 'WM-71803-12_250_300_50.datastack',
 'WM-71803-13_400_600_50.datastack',
 'WM-71803-17_400_600_50.datastack',
 'WM-71803-18_400_300_50_det.datastack',
 'WM-71803-18_400_500_50.datastack',
 'WM-71803-19_400_600_50.datastack',
 'WM-71803-23_400_600_50.datastack',
 'WM-71803-24_400_600_50.datastack',
 'WM-71803-29_400_500_50.datastack',
 'WM-71803-30_250_300_50.datastack',
 'WM-71803-31_400_600_50.datastack',
 'WM-71803-35_400_500_50.datastack']

`Hi Koen, would be nice to implement fsspec based preview thumbnails here.`

Now it's up to you to decide which files you want to download from our cloud storage. You can download individual files using the `.download_file(file_name)` method like so: 

In [ ]:
maxrf_data.download_file('WM-71803-01_400_600_50.datastack') 

Finished downloading


Or download all the datastack files using the `.download_all()` method. 

In [ ]:
maxrf_data.download_all() # Warning: 62.8 GB!

Checked 1/16 files
Checked 2/16 files


::: {.callout-warning}
The datastacks used for this project are quite big in size, totaling **62.8 GB**. Make sure you have enough space on your device, since this download will take a while.
:::

After downloading all files you might be curious as to where they are stored. Normally they are all saved in the .cache folder within your JupyterLab/Hub files on your device.
But if you want to know the exact location you can run the `.get_base_path()` method to see the exact location. 

In [ ]:
maxrf_data.get_base_path()

PosixPath('/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks')

For easy accesibility of the datastack files which are used all across this package you can use `downloader.get_datastack_files()`. This returns a list of file locations which you can use in all of the code requiring datastacks.

In [ ]:
datastack_files = maxrf_data.get_datastack_files()
datastack_files

['/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-01_400_600_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-03_250_300_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-08_250_300_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-10_250_300_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-12_250_300_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-13_400_600_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-17_400_600_50.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project/data/interim/maxrf/datastacks/WM-71803-18_400_300_50_det.datastack',
 '/media/frank/lonprojecten/2024-811_falnama-project

# FUNCTIONS

In [ ]:
#|export
import pooch
import tqdm
from glob import glob

In [ ]:
#|export
class Falnama_maxrf_data: 
    '''Use this class to download MA-XRF data to download and cache the '.datastack' files onto your local computer once. 

    In subsequent sessions you can conveniently access this data for further processing. 

    WARNING: 
    The total amount of MA-XRF data is quite large (62.8 GB).
    Depending on your internet bandwidth it might take an hour or longer to download them. 
    
    '''
    def __init__(self):
        '''Creates the downloader object, use this to call it's methods.'''
        self.downloader = pooch.create(path=pooch.os_cache("falnama"),
                    base_url="https://f002.backblazeb2.com/file/falnama-data/datastacks/",
                    registry={"WM-71803-01_400_600_50.datastack" : "ee2af34dcbea270de797003500400e7f0655654838bafc880d0943a440283411",
                            "WM-71803-03_250_300_50.datastack" : "ad5bff448f32d4ac4dac46ae6e516af1c96f64e9ced17e3021124f21a5dd4669",
                            "WM-71803-08_250_300_50.datastack" : "14b6ac5fc8acad456ad39b0dd0acbaf6fd9655732a1d11c437fcc0652b753996",
                            "WM-71803-10_250_300_50.datastack" : "5e76db5bfc4b764178e824a7fc6caf5f3e49e64a329dc92691673b5accd0f430",
                            "WM-71803-12_250_300_50.datastack" : "cf0d12cda57a2f348abe1cf29f450131d97e4845d78da20d270a340e9e103db7",
                            "WM-71803-13_400_600_50.datastack" : "e67fd574fa2be48dfaa4cd08feb592c560575a0956cb84d7307760d8de7674c2",
                            "WM-71803-17_400_600_50.datastack" : "83d978ab426844842318cc3a8daf32ab749b1fe81657ca455e0289e94b9ffcf9",
                            "WM-71803-18_400_300_50_det.datastack" : "97bc254b0848172501bf4a96ebf8bd7862e1285607e060bd37cb973cff556226",
                            "WM-71803-18_400_500_50.datastack" : "93f21e5a078f9091a9139ffe8446003ab4eacfcd579221ecccd63b31220dcc27",
                            "WM-71803-19_400_600_50.datastack" : "b0d57800439b2824dfa148510728d18b195167d6f0b38c0539cf11b9bf3176b1",
                            "WM-71803-23_400_600_50.datastack" : "c76b90cd11096b364e7f10fcda67b61459c6ce0bdec0e816d7cc3d4ec70a238f",
                            "WM-71803-24_400_600_50.datastack" : "964603613e2882f570472357a303de232f0afe6a82de692c4ec7d78cb765bd4a",
                            "WM-71803-29_400_500_50.datastack" : "82647269b4b9361f9596f327e64f3b17a1b0512628eb9ed82c344edc796ae26c",
                            "WM-71803-30_250_300_50.datastack" : "af8e1bd23b5cec8e4e710dce050a6ade6bd94660d605650cc6bf27b1fd7d5076",
                            "WM-71803-31_400_600_50.datastack" : "99a747ddf664263f4b1743db0e13657a2e6313fd6aed1088df47c691b6a40c75",
                            "WM-71803-35_400_500_50.datastack" : "9742160af6b274fed19dd5e1b1ab7ef4ffe7c430e9063ee3b0924bb17309a42e"},
                    env='FALNAMA_CACHE')
    
    def show_registry_files(self):
        '''Prints all files currently in the downloader registry.'''
        return self.downloader.registry_files

    def download_file(self, file_name):
        '''Uses Pooch.fetch to download a file from the registry.'''
        self.downloader.fetch(file_name, progressbar=True)
        print('Finished downloading')
    
    def download_all(self):
        '''Downloads all files found in the downloaders registry.'''
        for i, datastack_path in enumerate(self.downloader.registry_files):
            total = len(self.downloader.registry_files)
            datastack_file = self.downloader.fetch(datastack_path, progressbar=True)
            print(f"Checked {i+1}/{total} files")
        print("All data is up to date!")
    
    def get_base_path(self):
        '''Returns the base path of where the downloader stores your files, almost always found in the jupyter notebook .cache folder.'''
        return self.downloader.abspath
    
    def get_datastack_files(self):
        '''Looks for all files ending in .datastack and returns the list of paths where these are located.'''
        datastack_files = sorted(glob(f'{self.downloader.abspath}/*.datastack', recursive=True))
        return datastack_files